### Open Source LLM

We can combine LLamaCPP or [Ollama integrated](https://ollama.ai) to use [Code LLaMA](https://about.fb.com/news/2023/08/code-llama-ai-for-coding/) /blog/run-code-llama-locally）.

Note：Please update `llama-cpp-python` , so that the `gguf` [format](https://github.com/abetlen/llama-cpp-python/pull/633) can be used here.

````
CMAKE_ARGS =“-DLLAMA_METAL = on”FORCE_CMAKE = 1 /Users/rlm/miniforge3/envs/llama2/bin/pip install -U llama-cpp-python --no-cache-dir
````

Check the lastest code-llama model [here](https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/tree/main).

In [5]:
!pip install -U llama-cpp-python
!pip install openai tiktoken chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 90.4 MB/s eta 0:00:00ta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 13.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.28-cp38-cp38-manylinux_2_31_x86_64.whl size=2113205 sha256=b4ce51d3f00900add3524f1b5ce69bc7602159f986c6ca4e7bc86ca7090370cb
  Stored in directory: /srv/users/rudxia/.cache/pip/wheels/12/12/f8/9e08a6fefe1455c9f86713e5c834fa8cfc117c64ca82c53c25
Successfully built llama-cpp-python


In [2]:
from langchain import PromptTemplate
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [8]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="/srv/users/rudxia/Developer_MICCAI/CodeLlama/model/codellama-13b-instruct.Q3_K_M.gguf",
    n_ctx=100000,
    n_gpu_layers=1,
    n_batch=512,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from /srv/users/rudxia/Developer_MICCAI/CodeLlama/model/codellama-13b-instruct.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = codellama_codellama-13b-instruct-hf
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_

## **Prompt after formatting**

In [38]:
question1="Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs."
prompt=PromptTemplate.from_template(question1)

default_chain=LLMChain(llm=llm, prompt=prompt, verbose=True)
default_chain.run(prompt=question1)



> Entering new LLMChain chain...
Prompt after formatting:
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs.


#include <iostream>
#include <iomanip>
using namespace std;
int main() {
	cout<<"Please enter the number:";
	double num, temp = 0;
	cin>>num;
	if (num % 2 == 0) {
		temp = num / 2;
	} else if ((num + 1) % 2 == 0)) {
		temp = ((num + 1))) / 2;
	}
	cout<<"The number is: " << temp << endl;





return 0;
















> Finished chain.



llama_print_timings:        load time =    2303.34 ms
llama_print_timings:      sample time =      41.32 ms /   154 runs   (    0.27 ms per token,  3727.01 tokens per second)
llama_print_timings: prompt eval time =    2301.16 ms /    64 tokens (   35.96 ms per token,    27.81 tokens per second)
llama_print_timings:        eval time =   21380.00 ms /   153 runs   (  139.74 ms per token,     7.16 tokens per second)
llama_print_timings:       total time =   24370.36 ms


'\n\n#include <iostream>\n#include <iomanip>\nusing namespace std;\nint main() {\n\tcout<<"Please enter the number:";\n\tdouble num, temp = 0;\n\tcin>>num;\n\tif (num % 2 == 0) {\n\t\ttemp = num / 2;\n\t} else if ((num + 1) % 2 == 0)) {\n\t\ttemp = ((num + 1))) / 2;\n\t}\n\tcout<<"The number is: " << temp << endl;\n\n\n\n\n\nreturn 0;\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [9]:
question2="Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Include the C++ standard library algorithm header for std::sort to ensure the program can sort the user inputs in ascending order based on the provided numeric input. \
Use a suitable data structure to efficiently store and manage the user input and timestamps records. \
Implement C++ standard library vector header. Include the C++ standard library ctime header for handling timestamps. \
Implement a function to display the sorted user input records with timestamps."

prompt=PromptTemplate.from_template(question2)

default_chain=LLMChain(llm=llm, prompt=prompt, verbose=True)
default_chain.run(prompt=question2)



> Entering new LLMChain chain...
Prompt after formatting:
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. Include the C++ standard library algorithm header for std::sort to ensure the program can sort the user inputs in ascending order based on the provided numeric input. Use a suitable data structure to efficiently store and manage the user input and timestamps records. Implement C++ standard library vector header. Include the C++ standard library ctime header for handling timestamps. Implement a function to display the sorted user input records with timestamps.
 Implement a function to exit the program from within the program.
### Sample Output ###

  -----------------------------------------------
    |                                                                |
      -------------------


llama_print_timings:        load time =    5543.33 ms
llama_print_timings:      sample time =      14.25 ms /    46 runs   (    0.31 ms per token,  3227.39 tokens per second)
llama_print_timings: prompt eval time =    5542.52 ms /   144 tokens (   38.49 ms per token,    25.98 tokens per second)
llama_print_timings:        eval time =    7069.67 ms /    45 runs   (  157.10 ms per token,     6.37 tokens per second)
llama_print_timings:       total time =   12815.91 ms /   189 tokens


' Implement a function to exit the program from within the program.\n### Sample Output ###\n\n  -----------------------------------------------\n    |                                                                |\n      --------------------------\n\n\n\n'

## **Prompt directly**

In [11]:
prompt1 = """
Question: Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs.
"""
llm(prompt1)

\

Llama.generate: prefix-match hit


begin{code}
#include <iostream>
using namespace std;
int main() {
    cout << "Enter an integer: ";
    int userInput = 0, tempIntVar = 0;
    cin >> userInput;
    if (userInput > 1) {
        cout << "\n";
        for (int counter = 2; counter <= userInput; ++counter) {
            cout << counter << " ";
            if (counter % 3 == 0)) {
                cout << endl;
            } else if ((counter >= 5) && (counter % 5 == 0))))))) {
                        cout << "\n";
                        for (int counter = 2; counter <= userInput; ++counter) {
                                    cout << counter << " ";
                                    if (counter % 3 == 0)) {
                                                            cout << endl;
                                                        } else if (((counter >= 5) && (counter % 5 == 0))) {
                                                                                                                cout << "\n";
       


llama_print_timings:        load time =    5911.12 ms
llama_print_timings:      sample time =      69.68 ms /   256 runs   (    0.27 ms per token,  3674.04 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   37557.97 ms /   256 runs   (  146.71 ms per token,     6.82 tokens per second)
llama_print_timings:       total time =   38747.29 ms /   257 tokens


'\\begin{code}\n#include <iostream>\nusing namespace std;\nint main() {\n    cout << "Enter an integer: ";\n    int userInput = 0, tempIntVar = 0;\n    cin >> userInput;\n    if (userInput > 1) {\n        cout << "\\n";\n        for (int counter = 2; counter <= userInput; ++counter) {\n            cout << counter << " ";\n            if (counter % 3 == 0)) {\n                cout << endl;\n            } else if ((counter >= 5) && (counter % 5 == 0))))))) {\n                        cout << "\\n";\n                        for (int counter = 2; counter <= userInput; ++counter) {\n                                    cout << counter << " ";\n                                    if (counter % 3 == 0)) {\n                                                            cout << endl;\n                                                        } else if (((counter >= 5) && (counter % 5 == 0))) {\n                                                                                                            

In [10]:
prompt2 = """
Question: Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Include the C++ standard library algorithm header for std::sort to ensure the program can sort the user inputs in ascending order based on the provided numeric input. \
Use a suitable data structure to efficiently store and manage the user input and timestamps records. \
Implement C++ standard library vector header. Include the C++ standard library ctime header for handling timestamps. \
Implement a function to display the sorted user input records with timestamps.
"""
llm(prompt2)

Llama.generate: prefix-match hit


Comment: Welcome to Stackoverflow. Please read [How do I ask a good question?](https://stackoverflow.com/help/how-to-ask) and update your question accordingly.
Answer: You have many mistakes in your code, but I will not write all of them here because it would be very long. But you can solve your problem by looking at the corrected version of your program below.
\begin{code}
#include <iostream>
using namespace std;
//function to sort user inputs based on their numeric value
void sortInputs(string &userInput1, string &userInput2, string &userInput3)){
  //define a variable called "sortedUserInputs" as an array of three strings.
   sortedUserInputs[0] = userInput1;
    sortedUserInputs[1] = userInput2;
     sortedUserInputs[2] = userInput3;
       //define a variable called "tempString" as a string.
            tempString = "";

                   //define a variable called "userInput1", "userInput2", and "userInput3" as three strings each that will store the numeric input provided by the


llama_print_timings:        load time =    5543.33 ms
llama_print_timings:      sample time =      73.37 ms /   256 runs   (    0.29 ms per token,  3489.35 tokens per second)
llama_print_timings: prompt eval time =    4788.57 ms /   148 tokens (   32.36 ms per token,    30.91 tokens per second)
llama_print_timings:        eval time =   38825.58 ms /   255 runs   (  152.26 ms per token,     6.57 tokens per second)
llama_print_timings:       total time =   44794.33 ms /   403 tokens


'Comment: Welcome to Stackoverflow. Please read [How do I ask a good question?](https://stackoverflow.com/help/how-to-ask) and update your question accordingly.\nAnswer: You have many mistakes in your code, but I will not write all of them here because it would be very long. But you can solve your problem by looking at the corrected version of your program below.\n\\begin{code}\n#include <iostream>\nusing namespace std;\n//function to sort user inputs based on their numeric value\nvoid sortInputs(string &userInput1, string &userInput2, string &userInput3)){\n  //define a variable called "sortedUserInputs" as an array of three strings.\n   sortedUserInputs[0] = userInput1;\n    sortedUserInputs[1] = userInput2;\n     sortedUserInputs[2] = userInput3;\n       //define a variable called "tempString" as a string.\n            tempString = "";\n\n                   //define a variable called "userInput1", "userInput2", and "userInput3" as three strings each that will store the numeric input

In [12]:
from langchain import PromptTemplate
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="/srv/users/rudxia/Developer_MICCAI/CodeLlama/model/codellama-13b-instruct.Q3_K_M.gguf",
    n_ctx=100000,
    n_gpu_layers=1,
    n_batch=512,
    f16_kv=True,  # MUST set to True, otherwise you will run into problems after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

question2 = "Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Include the C++ standard library algorithm header for std::sort to ensure the program can sort the user inputs in ascending order based on the provided numeric input. \
Use a suitable data structure to efficiently store and manage the user input and timestamps records. \
Implement C++ standard library vector header. Include the C++ standard library ctime header for handling timestamps. \
Implement a function to display the sorted user input records with timestamps."

# Ensure that the prompt contains placeholders for the library includes
prompt_with_includes = "#include <iostream>\n#include <vector>\n#include <algorithm>\n#include <ctime>\n" + question2

prompt_template = PromptTemplate.from_template(prompt_with_includes)

default_chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)
default_chain.run(prompt=prompt_with_includes)


llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from /srv/users/rudxia/Developer_MICCAI/CodeLlama/model/codellama-13b-instruct.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = codellama_codellama-13b-instruct-hf
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_



> Entering new LLMChain chain...
Prompt after formatting:
#include <iostream>
#include <vector>
#include <algorithm>
#include <ctime>
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. Include the C++ standard library algorithm header for std::sort to ensure the program can sort the user inputs in ascending order based on the provided numeric input. Use a suitable data structure to efficiently store and manage the user input and timestamps records. Implement C++ standard library vector header. Include the C++ standard library ctime header for handling timestamps. Implement a function to display the sorted user input records with timestamps.

#include <iostream>
int main() {
std::cout << "Hello World!" << std::endl;
return 0;
}
> Finished chain.



llama_print_timings:        load time =    5605.11 ms
llama_print_timings:      sample time =       8.73 ms /    33 runs   (    0.26 ms per token,  3781.37 tokens per second)
llama_print_timings: prompt eval time =    5602.53 ms /   169 tokens (   33.15 ms per token,    30.16 tokens per second)
llama_print_timings:        eval time =    5086.33 ms /    32 runs   (  158.95 ms per token,     6.29 tokens per second)
llama_print_timings:       total time =   10830.45 ms /   201 tokens


'\n#include <iostream>\nint main() {\nstd::cout << "Hello World!" << std::endl;\nreturn 0;\n}'